#### Import packages, set directories and parameters

In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from Bio import Entrez, SeqIO, AlignIO, pairwise2, Align, Seq, motifs
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
import math
from tqdm import tqdm
from Comparative_Analysis import Utilities as util
from joblib import Parallel, delayed
from Comparative_Analysis import Blast_Functions as blastfn
import time
import os
import copy
from scipy.stats import chi2, binom
from Comparative_Analysis import Alignment as alignfn
from Bio.Align.Applications import MuscleCommandline
import subprocess
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
project_dir = 'F:/Project_Data/mabR_Project'
mycobacteria_seq_dir = 'F:/Datasets/NCBI_Refseq_Mycobacteriaceae_Complete_Annot_20230511/data'
tb_species = 'AL123456.3' 
tb_annotation_dirname = 'GCA_000195955.2'
min_region_length = 7 
full_build = False
num_cores = 16
core_numbers = list(range(1, num_cores+1))
muscle_exe = 'C:/Users/nicho/Muscle/muscle3.8.31_i86win32.exe'
full_run = False

In [4]:
mycobacteria_dirs = []
for dir in util.list_dirs(mycobacteria_seq_dir):
    if os.path.exists(mycobacteria_seq_dir + '/' + dir + '/genomic.gbff'):
        mycobacteria_dirs.append(dir)

In [5]:
tb_mycobacteria_rbh = pd.read_csv(project_dir + '/tb_mycobacteria_reciprocal_best_hits.csv')

In [6]:
names_dict = {}
dirname_dict = {}
for dirname in (mycobacteria_dirs):
        for genome_record in SeqIO.parse(mycobacteria_seq_dir + '/' + dirname + '/genomic.gbff', "genbank"):
            accession_ver = genome_record.annotations['accessions'][0] + '.' + str(genome_record.annotations['sequence_version'])
            names_dict[accession_ver] = genome_record.annotations['organism']
        dirname_dict[dirname] = genome_record.annotations['organism']

#### Lookup function to find orthologs in M.tb based on reciprocal best hit outputs

In [7]:
def ortholog_in_mtb(accession_ver, locus):
    gene = accession_ver + '@' + locus
    temp = tb_mycobacteria_rbh.query('target_ref == @gene and query_species == @tb_species')
    if len(temp) == 1:
        for i, r in temp.iterrows():
            temp2 = r['query_ref']
            return temp2.split('@')[1]
    else:
        return 'No_orth'

#### Produce reference FASTA files (a) TB and (b) all mycobacteria for searching against motifs

In [8]:
genome_record = next(SeqIO.parse(mycobacteria_seq_dir + '/' + tb_annotation_dirname + '/genomic.gbff', "genbank"))
mtb_sequence = str(genome_record.seq)
accession_ver = genome_record.annotations['accessions'][0] + '.' + str(genome_record.annotations['sequence_version'])
util.produce_fasta_file([[accession_ver, mtb_sequence]], project_dir + '/mtb_seq')

100%|██████████| 1/1 [00:00<00:00, 11.65it/s]


In [9]:
if full_run == True:
    temp = []
    for dir in util.list_dirs(mycobacteria_seq_dir): 
        if os.path.exists(mycobacteria_seq_dir + '/' + dir + '/genomic.gbff'):
            for genome_record in (SeqIO.parse(mycobacteria_seq_dir + '/' + dir + '/genomic.gbff', "genbank")):
                accession_ver = genome_record.annotations['accessions'][0] + '.' + str(genome_record.annotations['sequence_version'])
                full_sequence = str(genome_record.seq)
                temp.append([accession_ver, full_sequence])
    util.produce_fasta_file(temp, project_dir + '/all_seq')

#### Functions to locate arbitrary region in organism

In [10]:
def annotated_regions_dataset(num_subsets, subset_num, dir_list, seqdir): 
    output = []
    sequence_dirs = util.chunk_list(dir_list, num_subsets, subset_num)
    for dirname in sequence_dirs:
        annotated_regions = []
        intergenic_regions = []
        for record in (SeqIO.parse(seqdir + '/'+dirname+'/genomic.gbff', "genbank")):
            accession_ver = record.annotations['accessions'][0] + '.' + str(record.annotations['sequence_version'])
            for feature in record.features:
                a = feature.qualifiers
                if feature.type not in ['source','gene'] and (int(feature.location.start) < int(feature.location.end)) and (int(feature.location.end) - int(feature.location.start)) < 1000000:
                    if not(a.get("product") == None):
                           product = a.get("product")[0]
                    if not(a.get("locus_tag")==None):
                        locus_tag = a.get("locus_tag")[0]
                    else:
                        locus_tag = feature.type
                    annotated_regions.append((locus_tag, product, feature.type, int(feature.location.start), int(feature.location.end), str(feature.location.strand)))
            annotated_regions.sort(key = lambda x: x[4])
            prev_strand = 0
            prev_locus = ''
            prev_product = ''
            max_stop = 0
            for n, (locus, product, feature_type, start, stop, strand) in enumerate(annotated_regions):
                if start > max_stop:
                    intergenic_regions.append([prev_locus+':'+locus, prev_product + ':' + product, 'Inter-feature',max_stop, start, str(prev_strand)+':'+str(strand)])
                if stop > max_stop:
                    prev_locus = locus
                    prev_product = product
                    prev_strand = strand
                max_stop = max(max_stop, stop)    
            for x in intergenic_regions:
                annotated_regions.append(x)
            annotated_regions.sort(key = lambda x : x[4])
            output.append([accession_ver, annotated_regions])
    return output

In [11]:
parallel_output = Parallel(n_jobs=-1)(delayed(annotated_regions_dataset)(num_cores, core_number, mycobacteria_dirs, mycobacteria_seq_dir) for core_number in core_numbers)
annotated_regions_dict = {}
for x in parallel_output:
    for y in x:
        annotated_regions_dict[y[0]] = y[1]

In [12]:
def location(accession_ver, start1, end1):
    feature_matches = []
    for (locus, product, feature, start, stop, strand) in annotated_regions_dict[accession_ver]:
            if start< end1 and stop > start1:
                overlap = str(int(100*(min(end1, stop) - max(start1, start))/ (end1-start1)))+'%'
                feature_matches.append([locus, product, feature, overlap, strand])
    return feature_matches

#### Create dictionary of upstream sequences - sequence continue upstream to the next CDS not next feature (e.g. if intervening repeat region this is included

In [13]:
def reverse_complement(seq_string):
    temp = []
    for char in reversed(seq_string):
        if char == 'A':
            comp = 'T'
        elif char == 'T':
            comp = 'A'
        elif char == 'C':
            comp = 'G'
        elif char == 'G':
            comp = 'C'
        else:
            comp = char
        temp.append(comp)
    return ''.join(temp)

In [40]:
def generate_upstream_sequence_dataset(num_subsets, subset_num, dir_list, seqdir, additional_offset = 0):     # Set additional offset = 3 if you want start codon of downstream CDS to be included - useful for alignment 
    sequence_dirs = util.chunk_list(dir_list, num_subsets, subset_num)
    upstream_cds_regions = []
    for dirname in (sequence_dirs):
        all_features = []
        cds_features = []
        for genome_record in SeqIO.parse(mycobacteria_seq_dir + '/' + dirname + '/genomic.gbff', "genbank"):
            accession_ver = genome_record.annotations['accessions'][0] + '.' + str(genome_record.annotations['sequence_version'])
            full_sequence = str(genome_record.seq)
            len_full_sequence = len(full_sequence)
            for feature in genome_record.features:
                if feature.type in ['gene', 'source']:
                    continue
                a = feature.qualifiers
                feature_type = feature.type
                if a.get("locus_tag") != None:
                    locus_tag = a.get("locus_tag")[0]
                    accession_locus = accession_ver + '@' + locus_tag
                else:
                    accession_locus  = ''
                (start, stop, strand) = (int(feature.location.start), int(feature.location.end), int(feature.location.strand))
                all_features.append([accession_locus, feature_type, start, stop, strand])
                if feature_type == 'CDS':
                    cds_features.append([accession_locus, feature_type, start, stop, strand])
            
            features = cds_features  # Replace with line below if you want to go upstream only to next annotated feature and not next CDS
            #features = all_features
            
            # Positive strand upstream 
            features.sort(key = lambda x: x[2])
            max_stop = 0
            for (accession_locus, feature_type, start, stop, strand) in features:
                if max_stop < start and feature_type == 'CDS' and strand == 1 and start - max_stop < 100000:    #Avoid joins where biopython interprets inconsistently 
                    upstream_cds_regions.append([accession_locus, max_stop, start, strand, full_sequence[max_stop: start+additional_offset]])
                max_stop = max(max_stop, stop)
            # Negative strand upstream
            features.sort(key = lambda x: x[3], reverse = True)
            min_start = len(full_sequence)-1
            for (accession_locus, feature_type, start, stop, strand) in features:
                if stop < min_start and feature_type == 'CDS' and strand == -1 and min_start - stop < 100000:
                    upstream_cds_regions.append([accession_locus, stop, min_start, strand, reverse_complement(full_sequence[stop-additional_offset: min_start])])
                min_start = min(min_start, start)
    return (upstream_cds_regions)           

In [14]:
if full_build == True:
#if 1==1:
    mycobacteria_upstream_dict = {}
    parallel_output = Parallel(n_jobs=-1)(delayed(generate_upstream_sequence_dataset)(num_cores, core_number, mycobacteria_dirs, mycobacteria_seq_dir) for core_number in core_numbers)
    for x in parallel_output:
        for n in x:
            mycobacteria_upstream_dict[n[0]] = [n[1], n[2], n[3], n[4]]
    with open(project_dir + '/mycobacteria_upstream_dict.pkl', 'wb') as f:
        pickle.dump(mycobacteria_upstream_dict, f) 
else:
    with open(project_dir + '/mycobacteria_upstream_dict.pkl', 'rb') as f:
        mycobacteria_upstream_dict = pickle.load(f)    

#### Functions to run MEME and FIMO

In [15]:
def run_meme(search_regions_loc, output_dir, min_width, min_sites):
    subprocess.run('wsl export PATH=$HOME/meme/bin:$HOME/meme/libexec/meme-5.4.1:usr/bin:$PATH ; meme '+ util.wslname(search_regions_loc) + ' -oc '+ util.wslname(output_dir) +' -dna -evt 0.001 -p 8 -revcomp -mod anr -brief 4000 -minw ' + str(min_width) +' -maxw 200 -minsites ' + str(min_sites)
               , shell=True)

In [16]:
def run_fimo(motif_file, sequence_to_search_file, output_dir):
    subprocess.run('wsl export PATH=$HOME/meme/bin:$HOME/meme/libexec/meme-5.4.1:usr/bin:$PATH ; fimo -oc ' + util.wslname(output_dir) + ' ' + util.wslname(motif_file) + ' ' + util.wslname(sequence_to_search_file)
               , shell=True)

In [17]:
def produce_matched_sequence_file(hit_df, output_file, q_value):
    matched_sequences = []
    for i, r in hit_df.iterrows():
        if '#' in r['motif_id']:
            continue
        if float(r['q-value']) < q_value:
            matched_sequences.append([str(r['start'])+'_'+str(r['stop']), r['matched_sequence']])
    util.produce_fasta_file(matched_sequences, output_file)

#### Import list of significantly up/downregulated genes and produce fasta file of upstream sequences

In [18]:
regulation_type_dict = {}
p_val_dict = {}
downreg_genes = pd.read_excel(project_dir + '/2022-10-02_data_NU_NAs_renamed.xlsx', sheet_name = 'significant genes downregulated')
upreg_genes = pd.read_excel(project_dir + '/2022-10-02_data.xlsx', sheet_name = 'significant genes upregulated')
all_genes = pd.read_excel(project_dir + '/2022-10-02_data.xlsx', sheet_name = 'all genes')
gene_list = []
for i, r in downreg_genes.iterrows():
    gene_list.append(r['Locus'])
    regulation_type_dict[r['Locus']] = 'Downregulated'
for i, r in upreg_genes.iterrows():
    gene_list.append(r['Locus'])
    regulation_type_dict[r['Locus']] = 'Upregulated'
for i, r in all_genes.iterrows():
    if r['padj'] == 'NA':
        p_val_dict[r['Locus']] = 999
    else:    
        p_val_dict[r['Locus']] = r['padj']

In [100]:
significant_upstream_tb_regions = []
temp_dict = {}
for gene in gene_list:
    locus = tb_species + '@' + str(gene)
    if locus in mycobacteria_upstream_dict:
        upstream_region = mycobacteria_upstream_dict[locus][3]  
        (start, stop) = (mycobacteria_upstream_dict[locus][0],  mycobacteria_upstream_dict[locus][1])
        if (start, stop) in temp_dict:
            pass  # Eliminate upstream region if already there (e.g if neighbouring divergent pair)
        else:
            temp_dict[(start, stop)] = 1      
            if len(upstream_region) >= min_region_length:
                significant_upstream_tb_regions.append([locus, upstream_region])
util.produce_fasta_file(significant_upstream_tb_regions, project_dir + '/significant_upstream_tb_regions.faa')

100%|██████████| 175/175 [00:00<00:00, 87527.21it/s]


#### Run MEME, search against tb using FIMO and process hits into final MEME Run = Second_MEME_Upstream_Output

In [101]:
run_meme(project_dir + '/significant_upstream_tb_regions.faa',project_dir + '/MEME_Upstream_Output',3,5)

In [102]:
run_fimo(project_dir + '/MEME_Upstream_Output/meme.txt', project_dir + '/mtb_seq' , project_dir + '/FIMO_Upstream_Output')
fimo_upstream_hits = pd.read_csv(project_dir + '/FIMO_Upstream_Output/fimo.tsv', sep='\t')
produce_matched_sequence_file(fimo_upstream_hits, project_dir + '/fimo_upstream_hit_sequences.faa', q_value = 1e-3)
run_meme(project_dir + '/fimo_upstream_hit_sequences.faa', project_dir + '/Second_MEME_Upstream_Output',3,5)

100%|██████████| 110/110 [00:00<?, ?it/s]


##### Produce FIMO hits for TB and all mycobacteria

In [103]:
run_fimo(project_dir + '/Second_MEME_Upstream_Output/meme.txt', project_dir + '/mtb_seq' , project_dir + '/Second_FIMO_Upstream_Output')

In [146]:
run_fimo(project_dir + '/Second_MEME_Upstream_Output/meme.txt', project_dir + '/all_seq' , project_dir + '/All_Species_Second_FIMO_Upstream_Output')

#### Use all species hits as a template for MEME (All_Species_Output) and search these motifs in other species (then refine into motif and search again),  Hits in /Single_Species_MEME_FIMO Final motif in /Temp_Second_Meme_Upstream_Output, 

In [172]:
fimo_upstream_hits = pd.read_csv(project_dir + '/All_Species_Second_FIMO_Upstream_Output/fimo.tsv', sep='\t')
produce_matched_sequence_file(fimo_upstream_hits, project_dir + '/all_species_hit_sequences.faa', q_value = 1e-3)
run_meme(project_dir + '/all_species_hit_sequences.faa', project_dir + '/All_Species_Output',3,5)

In [ ]:
if full_run == True:
    for dirname in tqdm(mycobacteria_dirs):
        temp = []
        for genome_record in (SeqIO.parse(mycobacteria_seq_dir + '/' + dirname + '/genomic.gbff', "genbank")):
            ntsequence = str(genome_record.seq)
            accession_ver = genome_record.annotations['accessions'][0] + '.' + str(genome_record.annotations['sequence_version'])
            temp.append([accession_ver, ntsequence])
        util.produce_fasta_file(temp, project_dir + '/temp_seq')
        run_fimo(project_dir + '/All_Species_Output/meme.txt', project_dir + '/temp_seq' , project_dir + '/Single_Species_MEME_FIMO/' + dirname)
        fimo_upstream_hits = pd.read_csv(project_dir + '/Single_Species_MEME_FIMO/' + dirname + '/fimo.tsv', sep='\t')
        produce_matched_sequence_file(fimo_upstream_hits, project_dir + '/Single_Species_MEME_FIMO/' + dirname + '/temp_fimo_upstream_hit_sequences.faa', q_value = 1e-3)
        run_meme(project_dir + '/Single_Species_MEME_FIMO/' + dirname + '/temp_fimo_upstream_hit_sequences.faa', project_dir + '/Single_Species_MEME_FIMO/' + dirname + '/Temp_Second_MEME_Upstream_Output',3,5)

        run_fimo(project_dir + '/Single_Species_MEME_FIMO/' + dirname + '/Temp_Second_MEME_Upstream_Output/meme.txt', project_dir + '/temp_seq' , project_dir + '/Single_Species_MEME_FIMO/' + dirname)
        fimo_upstream_hits = pd.read_csv(project_dir + '/Single_Species_MEME_FIMO/' + dirname + '/fimo.tsv', sep='\t')
        produce_matched_sequence_file(fimo_upstream_hits, project_dir + '/Single_Species_MEME_FIMO/' + dirname + '/temp_fimo_upstream_hit_sequences.faa', q_value = 1e-3)
        run_meme(project_dir + '/Single_Species_MEME_FIMO/' + dirname + '/temp_fimo_upstream_hit_sequences.faa', project_dir + '/Single_Species_MEME_FIMO/' + dirname + '/Temp_Second_MEME_Upstream_Output',3,5)

#### Produce dictionary of hits for each species based on ortholog of hits in M.tb 

In [ ]:
appearance_dict = {}
for dirname in tqdm(mycobacteria_dirs):
    temp_set = set()
    temp_set_2 = set()
    temp_list_3 = []
    temp_fimo_hits = pd.read_csv(project_dir + '/Single_Species_MEME_FIMO/' + dirname + '/fimo.tsv', sep='\t')
    hit_dict = {}
    for i, r in temp_fimo_hits.iterrows():
        if '#' in r['motif_id']:
            continue
        organism = r['sequence_name']
        motif_id = r['motif_alt_id']
        if float(r['q-value']) < 1e-5:
            temp = location(r['sequence_name'],int(r['start']), int(r['stop']))
            ortholog_orientations = []
            for x in temp:
                if len(x[0].split(':')) == 1:
                    ortholog_orientations.append((ortholog_in_mtb(organism, x[0]), 'I'))
                else:
                    a = x[0].split(':')
                    b = x[4].split(':')
                    if b[0] == '1':
                        ortholog_orientations.append((ortholog_in_mtb(organism, a[0]), 'U'))
                    else:
                        ortholog_orientations.append((ortholog_in_mtb(organism, a[0]), 'D'))
                    if b[1] == '1':
                        ortholog_orientations.append((ortholog_in_mtb(organism, a[1]), 'D'))
                    else:
                        ortholog_orientations.append((ortholog_in_mtb(organism, a[1]), 'U'))
            for x in ortholog_orientations:
                temp_list_3.append(x)                                          
            temp2 = ([x[0].split(':') for x in temp])
            for x in temp2:
                temp_set = temp_set.union({ortholog_in_mtb(organism, w) for w in set(x)})
                temp_set_2 = temp_set_2.union(set(x))
    #print(temp_set)
    temp_set_3 = set(temp_list_3)
    
    for ortholog in temp_set_3:
        if ortholog in appearance_dict:
            appearance_dict[ortholog].append(dirname_dict[dirname])
        else:
            appearance_dict[ortholog] = [dirname_dict[dirname]]

In [ ]:
temp = []
for k, v in appearance_dict.items():
    temp.append((k[0], k[1], len(v), v))
temp.sort(key = lambda x: x[2], reverse = True)
for (x, y, l, m) in temp:
    print(x, y, l, len(m))

#### Output motifs per species

In [47]:
temp = []
from Bio import motifs
dirnames = ['GCA_010726505.1']
for dirname in mycobacteria_dirs:
    filename = project_dir + '/Single_Species_MEME_FIMO/'+dirname +'/Temp_Second_MEME_Upstream_Output/meme.xml'
    if os.path.exists(filename):
        with open(filename) as f:
            record = motifs.parse(f, 'MEME')
        for motif in record:
            if motif.id == 'motif_1':
                temp.append([dirname_dict[dirname].replace(' ','_').replace('[','').replace(']',''), str(motif.consensus)])
util.produce_fasta_file(temp, project_dir + '/unaligned_motifs.faa')
cline = MuscleCommandline(muscle_exe, input= project_dir + '/unaligned_motifs.faa', out = project_dir + '/aligned_motifs.faa')


100%|██████████| 66/66 [00:00<00:00, 65973.32it/s]


In [48]:
temp = util.read_fasta_to_array(project_dir + '/aligned_motifs.faa')

In [49]:
for dirname in mycobacteria_dirs:
    species = dirname_dict[dirname].replace(' ','_').replace('[','').replace(']','')
    if species in temp[0]:
        pass
    else:
        temp[0].append(species)
        temp[1].append('-'*len(temp[1][0]))
temp_2 = []
for (x, y) in zip(temp[0],temp[1]):
    temp_2.append([x,y])
util.produce_fasta_file(temp_2, project_dir + '/aligned_motifs_plus_blanks.faa')

100%|██████████| 100/100 [00:00<?, ?it/s]


#### Final motif and searches

In [58]:
#temp_fimo_hits = pd.read_csv(project_dir + '/Second_FIMO_Upstream_Output/fimo.tsv', sep='\t')
hit_dict = {}
for dirname in tqdm(mycobacteria_dirs):
    temp_fimo_hits = pd.read_csv(project_dir + '/Single_Species_MEME_FIMO/' + dirname + '/fimo.tsv', sep='\t')

    for i, r in temp_fimo_hits.iterrows():
        if '#' in r['motif_id']:
            continue
        organism = r['sequence_name']
        motif_id = r['motif_alt_id']
        if float(r['q-value']) < 1e-6 and motif_id == 'MEME-1':
            temp = location(r['sequence_name'],int(r['start']), int(r['stop']))
            if len(temp) > 0:
                temp.sort(key = lambda x: float(x[3].strip('%')), reverse = True)
                main_location = temp[0]
            else:
                main_location = ''
            info = [int(r['start']), int(r['stop']), r['strand'], float(r['q-value']), main_location, r['matched_sequence']]
            if organism in hit_dict:
                hit_dict[organism].append(info)
            else:
                hit_dict[organism] = [info]

for k, v in hit_dict.items():
    v.sort(key = lambda x: x[0])

100%|██████████| 100/100 [00:27<00:00,  3.69it/s]


In [59]:
genes = []
v = hit_dict[tb_species]
for (start, stop, strand, q, loc, matched_seq) in v:
    for x in loc[0].split(':'):
        genes.append(x)


In [60]:
b = set(genes)
inters = b.intersection(set(gene_list))
print(len(b), len(inters), len(gene_list))
print(1- binom.cdf(len(inters), len(b),(len(gene_list))/3900))

116 16 236
0.0006309547603230437


In [61]:
b.intersection(set(gene_list))

{'Rv0154c',
 'Rv0211',
 'Rv0467',
 'Rv0468',
 'Rv0711',
 'Rv0860',
 'Rv0896',
 'Rv1058',
 'Rv1180',
 'Rv2845c',
 'Rv2846c',
 'Rv3140',
 'Rv3319',
 'Rv3416',
 'Rv3792',
 'Rv3795'}

In [ ]:
for k, v in hit_dict.items():
    for x in v:
        print(x)

#### Number of motif hits in different species and output so can be merged with R to display info against phylo tree

In [63]:
hit_info = []
for k, v in hit_dict.items():
    temp = []
    for (start, stop, strand, q, loc, matched_seq) in v:
        for x in loc[0].split(':'):
            temp.append(x)
    hit_info.append([names_dict[k].replace(' ','_'),'Num_hits', len(set(temp))])
pd.DataFrame(hit_info, columns = ['name', 'variable', 'value']).to_csv(project_dir + '/Hit_Counts.csv')

#### Build phylo tree 

In [ ]:
max_len = len(tb_mycobacteria_rbh['target_species_name'].unique())

In [ ]:
full_ortholog_refs = []
temp = tb_mycobacteria_rbh.groupby('query_ref').agg({'target_ref': "count", 'percent_identical_matches': "min"}).reset_index()
temp = temp.query('target_ref == @max_len and percent_identical_matches > 85')
for i, r in temp.iterrows():
    full_ortholog_refs.append(r['query_ref'])

In [ ]:
def generate_protein_dataset(num_subsets, subset_num, dir_list, seqdir): 
    sequence_dirs = util.chunk_list(dir_list, num_subsets, subset_num)
    all_cds = []
    for dirname in (sequence_dirs):
        for genome_record in SeqIO.parse(seqdir + '/' + dirname + '/genomic.gbff', "genbank"):
            accession_ver = genome_record.annotations['accessions'][0] + '.' + str(genome_record.annotations['sequence_version'])
            full_sequence = str(genome_record.seq)
            for feature in genome_record.features:
                a = feature.qualifiers
                if feature.type == 'CDS' and a.get("translation") != None and a.get("locus_tag") != None:
                    locus_tag = a.get("locus_tag")[0]
                    accession_locus = accession_ver + '@' + locus_tag
                    translation = a.get("translation")[0]
                    all_cds.append([accession_locus, translation])
    return (all_cds)           

In [ ]:
full_build = True
if full_build == True:
    parallel_output = Parallel(n_jobs=-1)(delayed(generate_protein_dataset)(num_cores, core_number, mycobacteria_dirs, mycobacteria_seq_dir) for core_number in core_numbers)
    protein_dict = {}
    for x in parallel_output:
        for temp in x[0]:
            protein_dict[temp[0]] = temp[1]

In [ ]:
concatenated_alignment_dict = {}
for ref in tqdm(full_ortholog_refs):
    temp_seq = []
    temp = tb_mycobacteria_rbh[tb_mycobacteria_rbh['query_ref'] == ref]
    for i, r in temp.iterrows():
        temp_seq.append([r['target_species_name'].replace(' ','_'),protein_dict[r['target_ref']]])
    util.produce_fasta_file(temp_seq, project_dir +'/temp_seq.fasta')    
    cline = MuscleCommandline(muscle_exe, input= project_dir +'/temp_seq.fasta', out=project_dir +'/temp_seq_alignment.fasta')
    result = cline();    
    alignment = util.read_fasta_to_array(project_dir +'/temp_seq_alignment.fasta')    
    for (name, sequence) in zip(alignment[0], alignment[1]):
        if name in concatenated_alignment_dict:
            temp2 = concatenated_alignment_dict[name]
            concatenated_alignment_dict[name] = temp2 + sequence
        else:
            concatenated_alignment_dict[name] = sequence
temp  = []
for k, v in concatenated_alignment_dict.items():
    temp.append([k, v])
util.produce_fasta_file(temp, project_dir + '/concatenated_alignment.fasta')

In [323]:
mtb_sequence[2720643:2720656]

'GCAGTCGCAAAAG'

In [325]:
util.reverse_complement(mtb_sequence[2720640: 2720661])

'GGGGGCTTTTGCGACTGCTCA'